In [57]:
import numpy as np 
import pandas as pd 
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import patsy as pt
import sklearn.linear_model as lm
import random as rd

df = pd.read_csv('train.csv')
df = df.select_dtypes(include=['int64'])

X1  = df['OverallQual'].values
X1_scaled = (X1 - X1.mean())/X1.std()
X2  = df['GrLivArea'].values
X2_scaled = (X2 - X2.mean())/X2.std()
X3  = df['TotalBsmtSF'].values
X3_scaled = (X3 - X3.mean())/X3.std()
X4 = df['YearRemodAdd'].values
X4_scaled = (X4 - X4.mean())/X4.std()

y  = df['SalePrice'].values



w0 = 1
w1 = 1
w2 = 1 #GrLivArea
w3 = 1 #TotalBsmtSF
w4 = 1


L = 0.001  #шаг обновления весов
epochs = 10000 # кол-во итераций
lambd = 0

for i in range(epochs):
    w4_ = w4 + L * (sum(X4_scaled*(y - w0 - w1*X1_scaled - w2*X2_scaled - w3*X3_scaled - w4*X4_scaled)) + lambd * w4)
    w3_ = w3 + L * (sum(X3_scaled*(y - w0 - w1*X1_scaled - w2*X2_scaled - w3*X3_scaled - w4*X4_scaled)) + lambd * w3)
    w2_ = w2 + L * (sum(X2_scaled*(y - w0 - w1*X1_scaled - w2*X2_scaled - w3*X3_scaled - w4*X4_scaled)) + lambd * w2)
    w1_ = w1 + L * (sum(X1_scaled*(y - w0 - w1*X1_scaled - w2*X2_scaled - w3*X3_scaled - w4*X4_scaled)) + lambd * w1)
    w0_ = w0 + L * (sum((y - w0 - w1*X1_scaled - w2*X2_scaled - w3*X3_scaled - w4*X4_scaled)) + lambd * w0)
    w4 = w4_
    w3 = w3_
    w2 = w2_
    w1 = w1_
    w0 = w0_


X = np.array((X1_scaled, X2_scaled, X3_scaled, X4_scaled))
print("xtshape", X.T.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X.T, y, test_size=0.3, random_state=10)
print("X_train ", X_train.shape)
print("y_train ", y_train.shape)

X_train = X_train.T
X_test = X_test.T
print("xztrainshape", X_train.shape)
print("ztestshape", X_test.shape)

y_train_pred = w0 + w1*X_train[0] + w2*X_train[1] + w3 * X_train[2] + w4 * X_train[3]
y_test_pred = w0 + w1*X_test[0] + w2*X_test[1] + w3 * X_test[2] + w4 * X_test[3]


print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

test_df = pd.read_csv('test.csv')
res_df = pd.DataFrame(index = test_df['Id'])
data = np.round(w0 + w1*test_df['OverallQual'].values + w2*test_df['GrLivArea'].values + w3 * test_df['TotalBsmtSF'].values + w4 * test_df['YearRemodAdd'].values)
data = data.astype(int)

res_df['SalePrice']  = data
res_df.to_csv('res01.csv')

xtshape (782, 4)
X_train  (547, 4)
y_train  (547,)
xztrainshape (4, 547)
ztestshape (4, 235)
MSE train: 1823026245.250, test: 1266711057.746
R^2 train: 0.755, test: 0.801
